## Creacion de DataFrame para funciones

vamos a crear los DataFrame de datos auxiliares que serán necesarios para las funciones que vamos a desarrollar.

### Cargamos las librerias necesarias

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
import numpy as np
warnings.filterwarnings("ignore")
from fastapi.encoders import jsonable_encoder

### Cargamos los CSV necesarios para extraer la informacion necesaria

In [3]:
df_output = pd.read_csv("data_repo/output_steam_games_limpio.csv", encoding="utf-8")
df_reviews = pd.read_csv("data_repo/user_reviews_limpio.csv", encoding="utf-8")
df_items = pd.read_csv("data_repo/users_items_limpio.csv", encoding="utf-8")

In [4]:
df_output.columns

Index(['app_name', 'id', 'developer', 'price_numeric', 'release_year',
       'main_genre'],
      dtype='object')

In [5]:
df_reviews.columns

Index(['user_id', 'item_id', 'recommend', 'year_posted', 'sentiment_analysis'], dtype='object')

In [6]:
df_items.columns

Index(['user_id', 'items_count', 'item_id', 'item_name', 'playtime_forever',
       'playtime_hour'],
      dtype='object')

Se recopila la informacion de las CSV:
dataframe output (Informacion de juegos)
* app_name : Nombre del juego 
* id : Id de juego
* developer : desarrollador
* price_numeric : Precio de lanzamiento
* release_year : Año de lanzamiento
* main_genre : Genero principal

dataframe Reviews (Reseñas de juegos)
* user_id : Id de usuario
* item_id : Id de juego
* recommend : Si recomienda el juego
* year_posted : Año de recomendacion
* sentiment_analysis : Valorizacion ( 0-Negativo, 1-Neutral, 2-Positivo)

dataframe Items (Informacion de juegos por usuario)
* user_id : Id de usuario
* items_count : Cantidad de juegos por usuario
* item_id : Id del juego
* item_name : Nombre de juego
* playtime_forever : Tiempo jugado en minutos
* playtime_hour : Timpo juego en horas

### Funcion UserForGenre
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento

In [8]:
# se extrae columnas relevantes de df_output
userForGenre_output_subset = df_output[["id", "main_genre", "release_year"]]

In [9]:
# se extrae columnas relevantes de df_reviews
userForGenre_reviews_subset = df_reviews[["user_id", "item_id", "year_posted"]]

In [10]:
# se extrae columnas relevantes de df_items
userForGenre_items_subset = df_items[["user_id", "item_id", "playtime_hour"]]

In [11]:
# Se combina en un primer los dataframes
userForGenre_merged = userForGenre_items_subset.merge(userForGenre_reviews_subset, on=["user_id", "item_id"], how="inner")
userForGenre_merged

,user_id,item_id,playtime_hour,year_posted
0,76561197970982479,22200,4.516667,2011
1,76561197970982479,1250,166.766667,2011
2,76561197970982479,43110,13.900000,2011
3,js41637,227300,9.183333,2013
4,evcentric,107200,53.750000,2014
...,...,...,...,...
26452,76561198289386531,304930,34.600000,2013
26453,76561198289386531,413150,10.800000,2013
26454,GodLoveGuru,242760,44.533333,2013
26455,76561198296402247,280790,49.733333,2013


In [12]:
# Se combina todos en un solo dataframes
userForGenre_merged_2 = userForGenre_merged.merge(userForGenre_output_subset, left_on="item_id", right_on="id", how="inner")
userForGenre_merged_2

,user_id,item_id,playtime_hour,year_posted,id,main_genre,release_year
0,76561197970982479,22200,4.516667,2011,22200,Action,2009
1,seantheextraprawnsheepguy,22200,4.000000,2011,22200,Action,2009
2,pipekissXD,22200,11.500000,2013,22200,Action,2009
3,ClockworkLunatic,22200,0.383333,2015,22200,Action,2009
4,vschiffer,22200,4.900000,2012,22200,Action,2009
...,...,...,...,...,...,...,...
22818,K1NGCJS,294230,2.783333,2015,294230,Action,2014
22819,K1NGCJS,341310,4.350000,2015,341310,Adventure,2015
22820,laislabonita75,305920,0.266667,2015,305920,Adventure,2014
22821,llDracuwulf,307130,3.516667,2015,307130,Action,2014


In [13]:
# Agrupar por usuario, género principal y año de publicación, sumando las horas jugadas
df_userForGenre = userForGenre_merged_2.groupby(["user_id","main_genre","year_posted"])["playtime_hour"].sum().reset_index()
df_userForGenre

,user_id,main_genre,year_posted,playtime_hour
0,--000--,Action,2014,49.150000
1,--ace--,Action,2014,21.150000
2,--ionex--,Action,2015,14.033333
3,-Azsael-,Indie,2013,198.150000
4,-Beave-,Action,2014,47.733333
...,...,...,...,...
19124,zvanik,Action,2015,382.233333
19125,zynxgameth,Action,2013,64.100000
19126,zyr0n1c,Action,2013,40.966667
19127,zyr0n1c,Action,2014,10.716667


dataframe userForGenre:
* user_id : id de usuario
* main_genre : genero principal
* year_posted : año jugado
* playtime_hour : cantidad de horas juegadas

Guardamos el DataFrame como un CSV y un archivo PARQUET

In [14]:
userForGenre_limpio = "data_repo/df_UsersForGenre.csv"
df_userForGenre.to_csv(userForGenre_limpio, index=False, encoding="utf-8")
print(f"Se guardó el archivo {userForGenre_limpio}") 

Se guardó el archivo data_repo/df_UsersForGenre.csv


In [15]:
userForGenre_limpio_2 = "Datasets/df_UsersForGenre.parquet"
df_userForGenre.to_parquet(userForGenre_limpio_2, index=False)
print(f"Se guardó el archivo {userForGenre_limpio_2}")

Se guardó el archivo Datasets/df_UsersForGenre.parquet


## Funcion BestDeveloper
Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [16]:
# Se selecciona las columnas 'id' y 'developer' del DataFrame df_output y eliminar las filas duplicadas basadas en 'id'
df_output4 = df_output[["id", "developer"]].drop_duplicates()

# Se selecciona las columnas relevantes para las revisiones: 'item_id', 'recommend', 'year_posted' y 'sentiment_analysis'
df_reviews4 = df_reviews[["item_id", "recommend", "year_posted","sentiment_analysis"]]

# Se fusiona los DataFrames df_reviews4 y df_output4 en función de la coincidencia entre 'item_id' y 'id'
df_reviews_merged4 = pd.merge(df_reviews4, df_output4, left_on="item_id", right_on="id")

# Se filtra las revisiones fusionadas para seleccionar solo aquellas donde la recomendación es Falsa (False) y el análisis de sentimiento es igual a 0
negative_reviews4 = df_reviews_merged4[(df_reviews_merged4["recommend"]== False) & (df_reviews_merged4["sentiment_analysis"] == 0)]

# Se agrupa las revisiones negativas filtradas por año, item_id y desarrollador, contando el número de recomendaciones en cada grupo
grouped_reviews4 = negative_reviews4.groupby(["year_posted", "item_id", "developer"]).size().reset_index(name="recommendation_count")

# Se obtiene las tres filas con las mejores recomendaciones para cada año
df_bestDeveloper = (
    grouped_reviews4.sort_values(by=["year_posted", "recommendation_count"], ascending=[True, False])  # Cambiado a False para orden descendente
                   .groupby("year_posted")
                   .head(3)
                   .reset_index(drop=True)
)

df_bestDeveloper


,year_posted,item_id,developer,recommendation_count
0,2011,440,Valve,1
1,2011,18700,Broken Rules,1
2,2011,33460,Ubisoft Montpellier,1
3,2012,440,Valve,1
4,2012,42920,NeoCoreGames,1
5,2012,55110,Volition,1
6,2013,275850,Hello Games,52
7,2013,730,Valve,32
8,2013,440,Valve,25
9,2014,221100,Bohemia Interactive,19


dataframe bestDeveloper:
* year_posted : año jugado
* item_id : id de juego
* developer	: desarrollador
* recommendation_count : cantidad de recomendaciones

Guardamos el DataFrame como un CSV y un archivo PARQUET

In [17]:
bestDeveloper_limpio = "data_repo/df_BestDeveloper.csv"
df_bestDeveloper.to_csv(bestDeveloper_limpio, index=False, encoding="utf-8")
print(f"Se guardó el archivo {bestDeveloper_limpio}") 

Se guardó el archivo data_repo/df_BestDeveloper.csv


In [18]:
bestDeveloper_limpio_2 = "Datasets/df_BestDeveloper.parquet"
df_bestDeveloper.to_parquet(bestDeveloper_limpio_2, index=False)
print(f"Se guardó el archivo {bestDeveloper_limpio_2}")

Se guardó el archivo Datasets/df_BestDeveloper.parquet


## Funcion DeveloperReviews

Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [19]:
# Se seleciona las columnas relevantes de df_reviews
developerReviews_reviews = df_reviews[["user_id", "item_id","recommend","year_posted","sentiment_analysis"]]
developerReviews_reviews

,user_id,item_id,recommend,year_posted,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,1
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,1
...,...,...,...,...,...
57533,76561198312638244,70,True,2013,2
57534,76561198312638244,362890,True,2013,2
57535,LydiaMorley,273110,True,2013,1
57536,LydiaMorley,730,True,2013,1


In [20]:
# Se seleciona las columnas relevantes de df_output
developerReviews_output = df_output[["id","app_name","developer"]]
developerReviews_output

,id,app_name,developer
0,761140,Lost Summoner Kitty,Kotoshiro
1,643980,Ironbound,Secret Level SRL
2,670290,Real Pool 3D - Poolians,Poolians.com
3,767400,弹炸人2222,彼岸领域
4,772540,Battle Royale Trainer,Trickjump Games Ltd
...,...,...,...
28829,745400,Kebab it Up!,Bidoniera Games
28830,773640,Colony On Mars,"Nikita ""Ghost_RUS"""
28831,733530,LOGistICAL: South Africa,Sacada
28832,610660,Russian Roads,Laush Dmitriy Sergeevich


In [21]:
# Se combina los dataframes en base al "items id" y "ID" del juego
developerReviews_merge = pd.merge(developerReviews_reviews, developerReviews_output, left_on="item_id", right_on="id")
developerReviews_merge

,user_id,item_id,recommend,year_posted,sentiment_analysis,id,app_name,developer
0,76561197970982479,1250,True,2011,2,1250,Killing Floor,Tripwire Interactive
1,death-hunter,1250,True,2015,2,1250,Killing Floor,Tripwire Interactive
2,DJKamBer,1250,True,2013,1,1250,Killing Floor,Tripwire Interactive
3,diego9031,1250,True,2015,1,1250,Killing Floor,Tripwire Interactive
4,76561198081962345,1250,True,2014,1,1250,Killing Floor,Tripwire Interactive
...,...,...,...,...,...,...,...,...
48870,llDracuwulf,307130,True,2015,1,307130,Asteria,Legend Studio
48871,ChrisCoroner,209120,True,2013,1,209120,Street Fighter X Tekken,"Capcom U.S.A., Inc."
48872,MeloncraftLP,220090,True,2013,0,220090,The Journey Down: Chapter One,SkyGoblin
48873,MeloncraftLP,262850,True,2013,0,262850,The Journey Down: Chapter Two,SkyGoblin


In [22]:
# Seleccionamos las columnas deseadas y agrupamos por desarrollador y análisis de sentimiento
developerReviews_merge_2 = developerReviews_merge[["user_id","app_name","developer","recommend","year_posted","sentiment_analysis"]]
df_developerReviews = developerReviews_merge_2.groupby(["developer","sentiment_analysis"]).size().reset_index(name="sentiment_analysis_count")
df_developerReviews

,developer,sentiment_analysis,sentiment_analysis_count
0,07th Expansion,0,1
1,07th Expansion,1,2
2,"10th Art Studio,Adventure Productions",0,1
3,"10th Art Studio,Adventure Productions",1,1
4,10tons Ltd,1,1
...,...,...,...
3977,"インレ,Inre",1,4
3978,橘子班,0,1
3979,橘子班,1,2
3980,橘子班,2,1


dataframe developerReviews:
* developer	: desarrollador
* sentiment_analysis : tipo de valorizacion
* sentiment_analysis_count : cantidad de valorizacion

Guardamos el DataFrame como un CSV y un archivo PARQUET

In [23]:
developerReviews_limpio = "data_repo/df_DeveloperReviews.csv"
df_developerReviews.to_csv(developerReviews_limpio, index=False, encoding="utf-8")
print(f"Se guardó el archivo {developerReviews_limpio}") 

Se guardó el archivo data_repo/df_DeveloperReviews.csv


In [24]:
developerReviews_limpio_2 = "Datasets/df_DeveloperReviews.parquet"
df_developerReviews.to_parquet(developerReviews_limpio_2, index=False)
print(f"Se guardó el archivo {developerReviews_limpio_2}")

Se guardó el archivo Datasets/df_DeveloperReviews.parquet


## Funcion Developer

Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora

In [25]:
# Se seleciona las columnas relevantes de df_output 
developer_output_subset = df_output[["id", "developer","price_numeric","release_year"]]
developer_output_subset

,id,developer,price_numeric,release_year
0,761140,Kotoshiro,4.99,2018
1,643980,Secret Level SRL,0.00,2018
2,670290,Poolians.com,0.00,2017
3,767400,彼岸领域,0.99,2017
4,772540,Trickjump Games Ltd,3.99,2018
...,...,...,...,...
28829,745400,Bidoniera Games,1.99,2018
28830,773640,"Nikita ""Ghost_RUS""",1.99,2018
28831,733530,Sacada,4.99,2018
28832,610660,Laush Dmitriy Sergeevich,1.99,2018


In [26]:
# Se agrupa por release_year y developer.
grupo_por_year_dev = developer_output_subset.groupby(['release_year', 'developer'])

# Se cuenta el número de id en cada grupo y crear una nueva columna
cantidad_ids = grupo_por_year_dev['id'].count().reset_index(name='cantidad_id')

# Se calcula el porcentaje de price_numeric igual a 0 para cada grupo y crear una nueva columna
porcentaje_price_numeric_0 = grupo_por_year_dev.apply(lambda x: (x['price_numeric'] == 0).mean() * 100).reset_index(name='contenido free')

# Se combina los resultados en un solo DataFrame
df_developer= cantidad_ids.merge(porcentaje_price_numeric_0, on=['release_year', 'developer'])
df_developer



,release_year,developer,cantidad_id,contenido free
0,1983,Digital Leisure Inc.,1,0.0
1,1984,Digital Leisure Inc.,1,0.0
2,1984,"Ed Hobbs,Robert Crossfield",1,0.0
3,1985,"MicroProse Software, Inc",1,0.0
4,1986,Choose Multiple LLC,1,0.0
...,...,...,...,...
15019,2018,杭州分浪网络科技有限公司,1,100.0
15020,2019,Ninetales Studios,1,0.0
15021,2019,Nodding Heads Games,1,100.0
15022,2019,Poetic Justice Studios,1,100.0


dataframe Developer:
* release_year: año de lanzamiento
* developer	: desarrollador
* cantidad_id : cantidad de items
* contenido free : % de contenido free por año

Guardamos el DataFrame como un CSV y un archivo PARQUET

In [27]:
developer_limpio = "data_repo/df_Developer.csv"
df_developer.to_csv(developer_limpio, index=False, encoding="utf-8")
print(f"Se guardó el archivo {developer_limpio}") 

Se guardó el archivo data_repo/df_Developer.csv


In [28]:
developer_limpio_2 = "Datasets/df_Developer.parquet"
df_developer.to_parquet(developer_limpio_2, index=False)
print(f"Se guardó el archivo {developer_limpio_2}")

Se guardó el archivo Datasets/df_Developer.parquet


## Funcion UserData

Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de ítems

In [30]:
# se extrae columnas relevantes de df_output
userData_output_subset = df_output[["id", "price_numeric"]]
userData_output_subset


,id,price_numeric
0,761140,4.99
1,643980,0.00
2,670290,0.00
3,767400,0.99
4,772540,3.99
...,...,...
28829,745400,1.99
28830,773640,1.99
28831,733530,4.99
28832,610660,1.99


In [31]:
# se extrae columnas relevantes de df_reviews
userData_reviews_subset = df_reviews[["user_id", "item_id", "recommend"]]
userData_reviews_subset

,user_id,item_id,recommend
0,76561197970982479,1250,True
1,76561197970982479,22200,True
2,76561197970982479,43110,True
3,js41637,251610,True
4,js41637,227300,True
...,...,...,...
57533,76561198312638244,70,True
57534,76561198312638244,362890,True
57535,LydiaMorley,273110,True
57536,LydiaMorley,730,True


In [32]:
# se extrae columnas relevantes de df_items
userData_items_subset = df_items[["user_id", "item_id", "items_count"]]
userData_items_subset

,user_id,item_id,items_count
0,76561197970982479,20,277
1,76561197970982479,30,277
2,76561197970982479,40,277
3,76561197970982479,60,277
4,76561197970982479,3830,277
...,...,...,...
3002796,76561198329548331,304930,7
3002797,76561198329548331,346330,7
3002798,76561198329548331,373330,7
3002799,76561198329548331,521570,7


In [51]:
userData_merged = userData_items_subset.merge(userData_output_subset, left_on="item_id", right_on="id", how="inner")
userData_merged

,user_id,item_id,items_count,id,price_numeric
0,76561197970982479,20,277,20,4.99
1,js41637,20,888,20,4.99
2,Riot-Punch,20,328,20,4.99
3,doctr,20,541,20,4.99
4,NitemarePK,20,304,20,4.99
...,...,...,...,...,...
2466257,PlayDusty,381890,379,381890,9.99
2466258,AQuestionableCharacter,443700,397,443700,11.99
2466259,thekushcorner,433400,450,433400,14.99
2466260,76561198071808318,444770,10,444770,7.99


In [54]:
# Se crea un df auxiliar 
df_gasto_items_usuario = userData_merged.groupby(['user_id','items_count'])['price_numeric'].sum().reset_index()
df_gasto_items_usuario

,user_id,items_count,price_numeric
0,--000--,58,146.88
1,--ace--,44,115.87
2,--ionex--,23,59.96
3,-2SV-vuLB-Kg,68,232.66
4,-404PageNotFound-,149,748.61
...,...,...,...
66849,zzonci,5,19.98
66850,zzoptimuszz,61,4.99
66851,zzydrax,13,19.99
66852,zzyfo,84,285.82


In [55]:
df_recommend = userData_reviews_subset.groupby('user_id')['recommend'].sum().reset_index()
df_recommend

,user_id,recommend
0,--000--,1
1,--ace--,2
2,--ionex--,2
3,-2SV-vuLB-Kg,5
4,-Azsael-,1
...,...,...
25133,zwanzigdrei,1
25134,zy0705,1
25135,zynxgameth,1
25136,zyr0n1c,9


In [59]:
userData_merged_2 = df_gasto_items_usuario.merge(df_recommend, on = 'user_id', how="inner")
userData_merged_2

,user_id,items_count,price_numeric,recommend
0,--000--,58,146.88,1
1,--ace--,44,115.87,2
2,--ionex--,23,59.96,2
3,-2SV-vuLB-Kg,68,232.66,5
4,-Azsael-,167,1104.41,1
...,...,...,...,...
22236,zwanzigdrei,93,431.68,1
22237,zy0705,6,0.00,1
22238,zynxgameth,50,279.75,1
22239,zyr0n1c,73,352.75,9


In [57]:
userData_merged_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22241 entries, 0 to 22240
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        22241 non-null  object 
 1   items_count    22241 non-null  int64  
 2   price_numeric  22241 non-null  float64
 3   recommend      22241 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 695.2+ KB


dataframe UserData:
* items_count : cantidad de items
* user_id : id de usuario
* price_numeric	: dinero gastado
* recommend	: cantidad de recomendaciones

Guardamos el DataFrame como un CSV y un archivo PARQUET

In [60]:
userData_limpio = "data_repo/df_UserData.csv"
userData_merged_2.to_csv(userData_limpio, index=False, encoding="utf-8")
print(f"Se guardó el archivo {userData_limpio}") 

Se guardó el archivo data_repo/df_UserData.csv


In [61]:
userData_limpio_2 = "Datasets/df_UserData.parquet"
userData_merged_2.to_parquet(userData_limpio_2, index=False)
print(f"Se guardó el archivo {userData_limpio_2}")

Se guardó el archivo Datasets/df_UserData.parquet
